# WuTong V5 Kaggle 一键训练

这个 Notebook 面向 Kaggle：
- 自动准备代码与依赖
- 自动从 `/kaggle/input` 扫描数据
- 后台启动 V5 训练（可断开页面继续）
- 实时查看日志、完成后打包模型

建议：直接 **Run All**。


In [ ]:
# 1) 克隆/更新代码到 /kaggle/working/WuTong
from pathlib import Path
import os

repo_dir = Path('/kaggle/working/WuTong')
if repo_dir.exists():
    print('Repo exists, pulling latest...')
    os.system(f'cd {repo_dir} && git pull')
else:
    print('Cloning repo...')
    os.system('git clone https://github.com/alltobebetter/WuTong.git /kaggle/working/WuTong')

%cd /kaggle/working/WuTong
!git log --oneline -n 2


In [ ]:
# 2) 安装依赖 + 检查环境
!pip -q install -r requirements.txt

import torch
print('CUDA available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('GPU:', torch.cuda.get_device_name(0))


In [ ]:
# 3) 从 /kaggle/input 拷贝数据到项目 data/raw
from pathlib import Path
import shutil

raw_dir = Path('data/raw')
raw_dir.mkdir(parents=True, exist_ok=True)

candidates = list(Path('/kaggle/input').rglob('*.xlsx')) + list(Path('/kaggle/input').rglob('*.csv'))
print(f'Found {len(candidates)} candidate files in /kaggle/input')
for p in candidates[:20]:
    print(' -', p)

for src in candidates:
    dst = raw_dir / src.name
    if not dst.exists():
        shutil.copy2(src, dst)

print('data/raw files:')
for p in raw_dir.glob('*'):
    print(' -', p)


In [ ]:
# 4) 一键预处理 + 增强 + 外部数据整合
import glob, os

xlsx_files = glob.glob('data/raw/*.xlsx')
csv_files = glob.glob('data/raw/*.csv')
source_files = xlsx_files + csv_files
if not source_files:
    raise RuntimeError('data/raw 没有可用数据，请先在 Kaggle Add Data 后重试')

src = source_files[0]
print('Using source file:', src)
ret = os.system(f'python -u scripts/ingest.py "{src}"')
if ret != 0:
    raise RuntimeError('ingest 失败')

ret = os.system('python -u scripts/augment_data.py --target-size 30000 --ratio 2.5')
if ret != 0:
    raise RuntimeError('augment 失败')

ret = os.system('python -u scripts/integrate_csic2010.py')
if ret != 0:
    raise RuntimeError('integrate_csic2010 失败')

print('preprocess pipeline done')


In [ ]:
# 5) 前台训练（适合 Save and Run All）
# 注意：这会阻塞 notebook 直到训练完成（约 40-60 分钟）
# 如果想快速测试，可以添加 --no-cv 或减少 --cv-splits

import subprocess
import sys

print('='*80)
print('🚀 V5 训练开始（前台模式，适合 Run All）')
print('='*80)
print('\n配置:')
print('  - 模型版本: v5.0.0-kaggle')
print('  - 交叉验证: 10 折')
print('  - 外部数据集: CSIC 2010')
print('  - SMOTE 过采样: 是')
print('  - 预计时间: 40-60 分钟\n')

cmd = [
    sys.executable, '-u', 'scripts/train_v5.py',
    '--version', 'v5.0.0-kaggle',
    '--cv-splits', '10'
]

# 实时输出训练日志
proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)
for line in proc.stdout:
    print(line, end='')

ret = proc.wait()
if ret != 0:
    raise RuntimeError(f'训练失败，退出码: {ret}')

print('\n' + '='*80)
print('✅ V5 训练完成！')
print('='*80)


In [ ]:
# 6) 查看训练结果
from pathlib import Path
import json

manifest_path = Path('models/v5.0.0-kaggle/manifest.json')

if manifest_path.exists():
    with open(manifest_path, 'r', encoding='utf-8') as f:
        manifest = json.load(f)

    print('='*80)
    print('📊 V5 训练结果')
    print('='*80)
    print(f"\n版本: {manifest['version']}")
    print(f"训练时间: {manifest['trained_at']}")
    print(f"数据量: {manifest['data_rows']} 条")
    print(f"外部数据集: {'是' if manifest.get('external_data') else '否'}")
    if manifest.get('external_data'):
        print(f"外部数据量: {manifest.get('external_data_rows', 0)} 条")
    print(f"特征数: {len(manifest['feature_list'])} 个")
    print(f"类别数: {len(manifest['classes'])} 类")

    print('\n' + '='*80)
    print('🎯 模型性能')
    print('='*80)

    metrics = manifest['metrics']
    
    # 集成模型性能
    if 'ensemble' in metrics:
        e = metrics['ensemble']
        print(f"\n🏆 集成模型（最终模型）")
        print(f"  准确率: {e['test_accuracy']:.4f} ({e['test_accuracy']*100:.2f}%)")
        print(f"  F1 分数: {e['test_f1']:.4f}")
        
        if e['test_accuracy'] >= 0.998:
            print(f"\n  🎉🎉🎉 恭喜！达到 99.8% 目标！")
        elif e['test_accuracy'] >= 0.995:
            print(f"\n  🎉🎉 恭喜！达到 99.5% 目标！")
        elif e['test_accuracy'] >= 0.99:
            print(f"\n  🎉 恭喜！达到 99% 目标！")
    
    print('\n' + '='*80)
else:
    print('❌ 未找到训练结果文件！')


In [ ]:
# 7) 打包并准备下载
from pathlib import Path
import shutil

model_dir = Path('models/v5.0.0-kaggle')
if model_dir.exists():
    print('📦 打包模型文件...')
    archive = shutil.make_archive(
        '/kaggle/working/models_v5.0.0-kaggle',
        'zip',
        'models',
        'v5.0.0-kaggle'
    )
    print(f'✅ 打包完成: {archive}')
    print(f'\n文件大小: {Path(archive).stat().st_size / 1024 / 1024:.2f} MB')
    print('\n💡 提示: 在 Kaggle 右侧 Output 面板可以下载此文件')
else:
    print('❌ 模型目录不存在！')


## 说明

### 训练模式

本 Notebook 使用**前台训练模式**，适合 Kaggle 的 "Save and Run All" 功能：
- ✅ 训练过程会实时显示在 notebook 中
- ✅ 训练完成后自动继续执行后续 cell
- ✅ 适合一次性完整训练
- ⏱️ 预计耗时：40-60 分钟（10折交叉验证）

### 快速测试

如果想快速测试（5-10分钟），可以修改第5个cell的命令：
```python
cmd = [
    sys.executable, '-u', 'scripts/train_v5.py',
    '--version', 'v5.0.0-kaggle-fast',
    '--no-cv'  # 禁用交叉验证
]
```

### 后台训练模式（不推荐用于 Run All）

如果需要后台训练（可以关闭页面），请手动执行以下命令：
```python
!python scripts/kaggle_bg_train_v5.py start --version v5.0.0-kaggle --cv-splits 10
!python scripts/kaggle_bg_train_v5.py tail -n 100  # 查看日志
```

注意：后台模式不适合 "Save and Run All"，因为 kernel 退出会杀死后台进程。
